In [1]:
%load_ext autoreload
%autoreload 2

Do ensure you have a vllm deployment ready! In my case I used notebook `2a. setup_vllm.ipynb` in google colab for my deployment

## Using OpenAI

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url="https://9398efd88ee3.ngrok-free.app/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="meta-llama/Llama-3.2-3B-Instruct",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None)


## Using LangChain

In [8]:
from langchain_community.llms import VLLMOpenAI

llm = VLLMOpenAI(
    openai_api_key="token-abc123",
    openai_api_base="https://9398efd88ee3.ngrok-free.app/v1",
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    model_kwargs={"stop": ["."]},
)
print(llm.invoke("Rome is"))

 a city that has been around for over 2,500 years


OH NO! `vLLMOpenAI` has no tool binding capabilities!

In [15]:
from langgraph.prebuilt import create_react_agent

def add(a: int | float, b: int | float) -> str:
    """Use this tool to add two integers or floats."""
    return a+b

vllm_agent = create_react_agent(llm, tools=[add])

AttributeError: 'VLLMOpenAI' object has no attribute 'bind_tools'

Use `ChatOpenAI` instead

In [10]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    model="meta-llama/Llama-3.2-3B-Instruct",
    temperature=0,
    api_key="token-abc123",
    base_url="https://9398efd88ee3.ngrok-free.app/v1",
)

In [16]:
openai_agent = create_react_agent(llm_openai, tools=[add])

In [18]:
async for chunk in openai_agent.astream(
    {"messages": [{"role": "user", "content": "what is 7+21?"}]},
    stream_mode="updates"
):
    # chunk['messages'].pretty_print()
    print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-0aa133f80ab94d1198ba8468f538aadf', 'function': {'arguments': '{"a": 7, "b": 21}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 308, 'total_tokens': 330, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-3.2-3B-Instruct', 'system_fingerprint': None, 'id': 'chatcmpl-61a08f6ffba141bab406033572f012eb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--680a53dd-3c48-4fc2-8440-36d16e48f055-0', tool_calls=[{'name': 'add', 'args': {'a': 7, 'b': 21}, 'id': 'chatcmpl-tool-0aa133f80ab94d1198ba8468f538aadf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 308, 'output_tokens': 22, 'total_tokens': 330, 'input_token_details': {}, 'output_token_details': {}})]}}
{'tools': {'messages': [ToolMessage(content='28', name='add', id='6